In [6]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 4.3 MB/s eta 0:00:0000:0100:01
  Using cached pytz-2022.7-py2.py3-none-any.whl (499 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 3.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
# import modules

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
from urllib.parse import urlencode
import base64
import webbrowser
import pandas as pd

In [8]:
# get client id, client secret from https://developer.spotify.com/dashboard/applications. spotify name is *your spotify name*. redirect_uri can be anything (used for getting auth)

client_id = ''
client_secret = ''
spotify_name = '11174931182'
redirect_uri = 'https://github.com/solidliquidity/'

In [9]:
# defining functions

# useful links for the get authorisation malarkey
# https://prettystatic.com/automate-the-spotify-api-with-python/
# https://lvngd.com/blog/accessing-spotify-api-python/#:~:text=Get%20an%20access%20token&text=To%20get%20authorized%2C%20we%20have,URL%20with%20our%20access%20credentials.&text=That's%20the%20authorization%20URL%2C%20and,to%20send%20with%20the%20request.&text=And%20send%20the%20request.&text=If%20all%20went%20well%2C%20the,should%20be%20in%20the%20response.

def get_access_token(client_id, client_secret):

    auth_url = 'https://accounts.spotify.com/api/token'
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    }
    
    auth_response = requests.post(auth_url, data=data)
    access_token = auth_response.json().get('access_token')

    return access_token

def get_playlists(spotify_name, offset, limit, auth_token):
    request = requests.get('https://api.spotify.com/v1/users/{spotify_name}/playlists?offset={offset}&limit={limit}'.format(spotify_name = spotify_name,
                                                                                                                            offset = offset,
                                                                                                                            limit = limit), 
                            headers={"Authorization": f"Bearer {auth_token}"}).json()
    return request

def get_playlist_items(spotify_name, id, offset, limit, auth_token):
    request = requests.get('https://api.spotify.com/v1/users/{spotify_name}/playlists/{id}/tracks?offset={offset}&limit={limit}'.format(spotify_name = spotify_name,
                                                                                                                                        id = id,
                                                                                                                                        offset = offset,
                                                                                                                                        limit = limit), 
                            headers={"Authorization": f"Bearer {auth_token}"}).json()
    return request

def get_track_features(spotify_name, id, auth_token):
    request = requests.get('https://api.spotify.com/v1/audio-features/{id}'.format(spotify_name = spotify_name, id = id), 
                            headers={"Authorization": f"Bearer {auth_token}"}).json()
    return request

def get_track_info(track_id, auth_token):
    
    request = requests.get('https://api.spotify.com/v1/tracks/{id}'.format(id = track_id), 
                            headers={"Authorization": f"Bearer {auth_token}"}).json()

    name = request['name']
    artists = request['artists'][0]['name']
    duration = request['duration_ms']
    popularity = request['popularity']
    release_date = request['album']['release_date']
    df = pd.DataFrame({'Name': name, 'Artists': artists, 'Duration': duration, 'Popularity': popularity, 'Release Date': release_date}, index = [0])

    return df


In [10]:
# 3 nested loops are you mad

themainframe = pd.DataFrame()

# get authorisation token

auth_token = get_access_token(client_id, client_secret)

# get playlists of user (limited to max 20 at a time)

playlists = get_playlists(spotify_name, 0, 20, auth_token)

# get the id of a playlist

# loop through each 20-limited set of playlists
z = 0 
while z < 1: 

    tracks_df = pd.DataFrame() # tracks dataframe to be filled

    # get playlists (limited to max 20 at a time)
    playlists = get_playlists(spotify_name, z, 20, auth_token)

    # get the id of each playlist


    y = 0
    while y < len(playlists):

        id = playlists['items'][y]['id'] # example id

        # get all of the tracks within the playlist (limited to max 50 at a time)

        tracks = get_playlist_items(spotify_name, id, 0, 50, auth_token)

        # get track id of each track
        
        x = 0
        while x < len(tracks['items']):
            
            track = tracks['items'][x]
            track_id = track['track']['id']

            # get track info

            track_info_df = get_track_info(track_id, auth_token)

            # get track audio features 

            track_features = get_track_features(spotify_name, track_id, auth_token)
            track_feature_df = pd.DataFrame(track_features, index = [0])

            # concatenate laterally & then append to the tracks dataframe from above

            track_df = pd.concat([track_info_df, track_feature_df], axis = 1)
            tracks_df = pd.concat([tracks_df, track_df])

            x += 1

        y += 1

    themainframe = pd.concat([themainframe, tracks_df])

    z += 1

In [11]:
themainframe = themainframe.sort_values(by = 'tempo')
themainframe

,Name,Artists,Duration,Popularity,Release Date,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Music for Six Rooms,Hoavi,606000,19,2021-11-12,0.0592,0.1930,5,-22.656,1,...,0.0976,0.0337,50.214,audio_features,2kTVyBg5HoJBXo2qNPb7Tf,spotify:track:2kTVyBg5HoJBXo2qNPb7Tf,https://api.spotify.com/v1/tracks/2kTVyBg5HoJB...,https://api.spotify.com/v1/audio-analysis/2kTV...,606001,4
0,Open Mind,gonima,155980,31,2022-06-24,0.1500,0.1600,5,-18.491,0,...,0.0892,0.0355,52.368,audio_features,2ffxaPccRoZmT99U5wgJQJ,spotify:track:2ffxaPccRoZmT99U5wgJQJ,https://api.spotify.com/v1/tracks/2ffxaPccRoZm...,https://api.spotify.com/v1/audio-analysis/2ffx...,155980,4
0,Lev,Nueen,274322,14,2022-10-28,0.2410,0.3540,3,-11.453,0,...,0.1130,0.1940,55.280,audio_features,02bmEXK0tsHEGENsoal7VA,spotify:track:02bmEXK0tsHEGENsoal7VA,https://api.spotify.com/v1/tracks/02bmEXK0tsHE...,https://api.spotify.com/v1/audio-analysis/02bm...,274323,4
0,Portable Void,Imaginary Softwoods,150000,33,2022-02-22,0.1990,0.0999,10,-21.121,1,...,0.1210,0.0379,57.691,audio_features,3M5LeKjUK2VKfWSD0p14M1,spotify:track:3M5LeKjUK2VKfWSD0p14M1,https://api.spotify.com/v1/tracks/3M5LeKjUK2VK...,https://api.spotify.com/v1/audio-analysis/3M5L...,150000,3
0,Mona Lisa,Israel Kamakawiwo'ole,120773,33,2017-10-15,0.5130,0.1410,5,-15.594,1,...,0.3770,0.4520,58.654,audio_features,5NLw0Fd6cixHeCXgqFXTa4,spotify:track:5NLw0Fd6cixHeCXgqFXTa4,https://api.spotify.com/v1/tracks/5NLw0Fd6cixH...,https://api.spotify.com/v1/audio-analysis/5NLw...,120773,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Sous Marin,Loscil,232320,37,2022-09-09,0.0822,0.0189,4,-37.728,1,...,0.1210,0.0250,172.862,audio_features,0qQAIBuM7K2CZs9dTuqZ1K,spotify:track:0qQAIBuM7K2CZs9dTuqZ1K,https://api.spotify.com/v1/tracks/0qQAIBuM7K2C...,https://api.spotify.com/v1/audio-analysis/0qQA...,232320,4
0,IWY,Submarine,294534,32,2019-02-25,0.5610,0.6320,1,-8.867,0,...,0.1320,0.0886,174.005,audio_features,1YIPDVF0X8dfIIiacuhg6D,spotify:track:1YIPDVF0X8dfIIiacuhg6D,https://api.spotify.com/v1/tracks/1YIPDVF0X8df...,https://api.spotify.com/v1/audio-analysis/1YIP...,294534,4
0,Luna,Submarine,282629,24,2018-11-02,0.6880,0.5470,2,-9.052,1,...,0.1180,0.0649,174.007,audio_features,6GhiMgNDzOAf2oDF9vXWnM,spotify:track:6GhiMgNDzOAf2oDF9vXWnM,https://api.spotify.com/v1/tracks/6GhiMgNDzOAf...,https://api.spotify.com/v1/audio-analysis/6Ghi...,282629,4
0,Vanta Black - Instrumental,Benny L,325517,18,2019-06-07,0.3130,0.8380,10,-2.950,1,...,0.1070,0.0352,174.007,audio_features,1qUUuwjhVDxDPUHbtXSYZ1,spotify:track:1qUUuwjhVDxDPUHbtXSYZ1,https://api.spotify.com/v1/tracks/1qUUuwjhVDxD...,https://api.spotify.com/v1/audio-analysis/1qUU...,325517,4
